<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/Momentum_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install yfinance

In [21]:
# import necessary packages
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_datareader.data as reader
import datetime as dt
from pandas.tseries.offsets import MonthEnd

In [22]:
symbols=pd.read_csv('https://raw.githubusercontent.com/anirbanghoshsbi/nifty500/main/ind_nifty500list.csv')

In [23]:
symbols['data']=symbols.Symbol+'.NS'


In [24]:
ticker=symbols.data.tolist()

In [25]:
# Get ticker symbol for NSE 50 stocks
#symbols=pd.read_html('https://in.finance.yahoo.com/quote/%5ENSEI/components?p=%5ENSEI')[0]
#ticker=symbols.Symbol.to_list()


In [26]:
#symbols=pd.read_html('https://www.moneycontrol.com/financials/tatasteel/balance-sheetVI/TIS#TIS')

In [27]:
#symbols[0]

In [28]:
#Get the NSE Component prices
start_date = dt.datetime(2018,1,31)
end_date= dt.datetime(2021,12,7)
df =yf.download(ticker,start_date,end_date)['Adj Close']

[*********************100%***********************]  501 of 501 completed

1 Failed download:
- ORIENTREF.NS: No data found, symbol may be delisted


In [29]:
# Get daily return and then cummulate it to monthly/Quartelry/Half yealrly /Annually return
monthly_return = df.pct_change().resample('M').agg(lambda x:(x+1).prod()-1)
#quarterly_return = df.pct_change().resample('Q').agg(lambda x:(x+1).prod()-1)
#halfyearly_return = df.pct_change().resample('H').agg(lambda x:(x+1).prod()-1)
#annual_return= df.pct_change().resample('Y').agg(lambda x:(x+1).prod()-1)


In [30]:
#Calculate returns over last 11 months to 6 months
past_11 = (monthly_return+1).rolling(11).apply(np.prod)-1
past_10 = (monthly_return+1).rolling(10).apply(np.prod)-1
past_9 = (monthly_return+1).rolling(9).apply(np.prod)-1
past_8 = (monthly_return+1).rolling(8).apply(np.prod)-1
past_7 = (monthly_return+1).rolling(7).apply(np.prod)-1
past_6 = (monthly_return+1).rolling(6).apply(np.prod)-1

In [31]:
#past_q_1= (quarterly_return+1).rolling(3).apply(np.prod)-1
#past_q_2= (quarterly_return+1).rolling(6).apply(np.prod)-1
#past_q_3= (quarterly_return+1).rolling(9).apply(np.prod)-1

In [32]:
formation=dt.datetime(2021,11,30)

In [33]:
from pandas.tseries.offsets import MonthEnd
end_measurement11= formation- MonthEnd(1)
end_measurement10 = formation-MonthEnd(2)

end_measurement09 = formation-MonthEnd(3)
end_measurement08 = formation-MonthEnd(4)
end_measurement07 = formation-MonthEnd(5)
end_measurement06 = formation-MonthEnd(6)

In [34]:
end_measurement11

Timestamp('2021-10-31 00:00:00')

In [35]:
past_11.tail(2)

,3MINDIA.NS,AARTIDRUGS.NS,AARTIIND.NS,AAVAS.NS,ABB.NS,ABBOTINDIA.NS,ABCAPITAL.NS,ABFRL.NS,ACC.NS,ADANIENT.NS,ADANIGREEN.NS,ADANIPORTS.NS,ADANITRANS.NS,ADVENZYMES.NS,AEGISCHEM.NS,AFFLE.NS,AIAENG.NS,AJANTPHARM.NS,AKZOINDIA.NS,ALEMBICLTD.NS,ALKEM.NS,ALKYLAMINE.NS,ALOKINDS.NS,AMARAJABAT.NS,AMBER.NS,AMBUJACEM.NS,ANGELBRKG.NS,APLAPOLLO.NS,APLLTD.NS,APOLLOHOSP.NS,APOLLOTYRE.NS,ASAHIINDIA.NS,ASHOKA.NS,ASHOKLEY.NS,ASIANPAINT.NS,ASTERDM.NS,ASTRAL.NS,ASTRAZEN.NS,ATGL.NS,ATUL.NS,...,TRENT.NS,TRIDENT.NS,TRITURBINE.NS,TTKPRESTIG.NS,TV18BRDCST.NS,TVSMOTOR.NS,UBL.NS,UCOBANK.NS,UFLEX.NS,UJJIVAN.NS,UJJIVANSFB.NS,ULTRACEMCO.NS,UNIONBANK.NS,UPL.NS,UTIAMC.NS,VAIBHAVGBL.NS,VAKRANGEE.NS,VALIANTORG.NS,VARROC.NS,VBL.NS,VEDL.NS,VENKEYS.NS,VGUARD.NS,VINATIORGA.NS,VIPIND.NS,VMART.NS,VOLTAS.NS,VSTIND.NS,VTL.NS,WABCOINDIA.NS,WELCORP.NS,WELSPUNIND.NS,WESTLIFE.NS,WHIRLPOOL.NS,WIPRO.NS,WOCKPHARMA.NS,YESBANK.NS,ZEEL.NS,ZENSARTECH.NS,ZYDUSWELL.NS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-11-30,0.201812,-0.281359,0.522199,0.660847,0.713492,0.221247,0.283529,0.540459,0.424116,2.469823,0.230667,0.423547,3.113143,-0.019455,-0.082506,0.518905,-0.034364,0.279290,-0.122354,-0.055428,0.240542,1.087818,-0.083691,-0.317478,0.287593,0.510700,2.291607,1.092865,-0.208193,1.358762,0.166132,0.750823,0.038316,0.266463,0.144128,0.157673,0.645264,-0.320409,3.260259,0.356834,...,0.476238,4.417585,1.315048,0.618932,0.333333,0.423581,0.268488,0.019685,0.307087,-0.509325,-0.536896,0.412597,0.362916,0.479562,1.010173,0.013332,-0.504088,-0.343253,-0.257549,0.464560,1.231798,0.433047,0.327173,0.563673,0.533882,0.552695,0.469277,-0.101047,0.922626,0.462904,0.288034,1.034020,0.201797,-0.221562,0.653547,-0.203092,-0.305322,0.472177,0.899905,-0.010993
2021-12-31,0.361068,-0.214731,0.655395,0.463366,0.490661,0.327450,0.450495,0.709527,0.392960,2.250158,0.371977,0.429308,3.072911,0.046414,-0.144443,0.462630,-0.066514,0.170919,-0.045362,0.064058,0.145531,0.658342,0.009412,-0.304645,0.241244,0.526492,2.248839,0.994485,-0.142108,1.070486,0.057085,0.752401,0.110412,0.086909,0.269839,0.214579,0.711943,-0.145309,3.097484,0.395057,...,0.606710,2.332569,1.197054,0.698843,0.457143,0.185913,0.154242,0.011673,0.435328,-0.438532,-0.466387,0.360891,0.441224,0.293619,0.870950,-0.005310,-0.345607,-0.223432,-0.321033,0.483035,1.213107,0.683111,0.018213,0.551693,0.554901,0.522975,0.364017,-0.062963,0.878126,0.504876,0.433241,1.097743,0.216667,-0.222706,0.494377,-0.109448,-0.200000,0.641791,0.852818,0.002888


In [36]:
ret_12 = past_11.loc[end_measurement11]

In [37]:
ret_11 = past_10.loc[end_measurement10]
ret_10 = past_9.loc[end_measurement09]
ret_09 = past_8.loc[end_measurement08]
ret_08 = past_7.loc[end_measurement07]
ret_07 = past_6.loc[end_measurement06]


In [38]:
ret_12=ret_12.reset_index()
ret_11=ret_11.reset_index()
ret_10=ret_10.reset_index()
ret_09=ret_09.reset_index()
ret_08=ret_08.reset_index()
ret_07=ret_07.reset_index()

In [39]:
ret_12['quintile'] = pd.qcut(ret_12.iloc[:,1],10,labels=False)
ret_11['quintile'] = pd.qcut(ret_11.iloc[:,1],10,labels=False)
ret_10['quintile'] = pd.qcut(ret_10.iloc[:,1],10,labels=False)
ret_09['quintile'] = pd.qcut(ret_09.iloc[:,1],10,labels=False)
ret_08['quintile'] = pd.qcut(ret_08.iloc[:,1],10,labels=False)
ret_07['quintile'] = pd.qcut(ret_07.iloc[:,1],10,labels=False)

In [40]:
winners12= ret_12[ret_12.quintile==9]
winners11= ret_11[ret_11.quintile==9]
winners10= ret_10[ret_10.quintile==9]
winners9= ret_09[ret_09.quintile==9]
winners08= ret_08[ret_08.quintile==9]
winners07= ret_07[ret_07.quintile==9]

In [41]:
# Getting Common Winners

from functools import reduce # python 3 only
final_winner=reduce(np.intersect1d, [winners12.index,winners11.index,\
                                      winners10.index,winners9.index,\
                                      winners08.index,winners07.index])

final_winner=final_winner.tolist()

In [ ]:
cols=[]
for ind in final_winner:
  print(df.columns[ind])
  cols.append(df.columns[ind])

In [ ]:
df[cols]